<a href="https://colab.research.google.com/github/alan713/alan1/blob/master/summarization_alan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
!pip install rank_bm25


In [ ]:
nltk.download("punkt")
nltk.download("punkt_tab")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
nltk.download("averaged_perceptron_tagger")


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
!pip uninstall -y nltk
!pip install nltk==3.8.1


Found existing installation: nltk 3.8.1
Uninstalling nltk-3.8.1:
  Successfully uninstalled nltk-3.8.1
  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
textblob 0.19.0 requires nltk>=3.9, but you have nltk 3.8.1 which is incompatible.


In [ ]:
import nltk
nltk.download("punkt")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import nltk
import string
import numpy as np
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display, clear_output

# Download NLTK models
nltk.download("punkt")

# Preprocess function
translator = str.maketrans("", "", string.punctuation)

def preprocess(text):
    return text.lower().translate(translator)

# TF-IDF based summarization
def summarize_with_tfidf(text, query, top_k=3):
    sentences = sent_tokenize(text)
    cleaned_sentences = [preprocess(s) for s in sentences]
    cleaned_query = preprocess(query)

    vectorizer = TfidfVectorizer().fit(cleaned_sentences + [cleaned_query])
    sentence_vectors = vectorizer.transform(cleaned_sentences)
    query_vector = vectorizer.transform([cleaned_query])

    similarities = cosine_similarity(query_vector, sentence_vectors).flatten()
    top_indices = similarities.argsort()[-top_k:][::-1]
    return [sentences[i] for i in top_indices]

# Widgets
text_input = widgets.Textarea(
    value=""".""",
    placeholder="Paste your document here...",
    description="Document:",
    layout=widgets.Layout(width='100%', height='150px')
)

query_input = widgets.Text(
    value="legal",
    placeholder="Enter your query (e.g. legal, privacy, medical)...",
    description="Query:",
    layout=widgets.Layout(width='50%')
)

slider_k = widgets.IntSlider(
    value=3,
    min=1,
    max=10,
    step=1,
    description='# Sentences:'
)

method_dropdown = widgets.Dropdown(
    options=['BM25', 'TF-IDF'],
    value='BM25',
    description='Method:'
)

run_button = widgets.Button(description="Summarize")
output = widgets.Output()

# Main callback
def on_click(b):
    with output:
        clear_output()
        text = text_input.value.strip()
        query = query_input.value.strip()
        k = slider_k.value
        method = method_dropdown.value

        if not text or not query:
            print("⚠️ Please enter both a document and a query.")
            return

        if method == 'TF-IDF':
            summary = summarize_with_tfidf(text, query, top_k=k)
        else:
            from rank_bm25 import BM25Okapi  # Lazy import
            sentences = sent_tokenize(text)
            tokenized_sentences = [preprocess(s).split() for s in sentences]
            tokenized_query = preprocess(query).split()
            bm25 = BM25Okapi(tokenized_sentences)
            scores = bm25.get_scores(tokenized_query)
            top_indices = np.argsort(scores)[-k:][::-1]
            summary = [sentences[i] for i in top_indices]

        print(f"\n🔍 Summary using {method} for query: '{query}' (Top {k} sentences)")
        for sent in summary:
            print(f"- {sent}")

run_button.on_click(on_click)

# Display UI
display(text_input, query_input, slider_k, method_dropdown, run_button, output)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Textarea(value='.', description='Document:', layout=Layout(height='150px', width='100%'), placeholder='Paste y…

Text(value='legal', description='Query:', layout=Layout(width='50%'), placeholder='Enter your query (e.g. lega…

IntSlider(value=3, description='# Sentences:', max=10, min=1)

Dropdown(description='Method:', options=('BM25', 'TF-IDF'), value='BM25')

Button(description='Summarize', style=ButtonStyle())

Output()

updated code

In [ ]:
# Paste this entire cell into Colab / Jupyter and run.

import nltk
import string
import numpy as np
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display, clear_output

# FIX: Download all necessary NLTK models
nltk.download("punkt")
nltk.download("punkt_tab")   # NEW requirement
nltk.download("averaged_perceptron_tagger")  # Occasionally required

# Preprocess function
translator = str.maketrans("", "", string.punctuation)


def preprocess(text):
    if text is None:
        return ""
    return text.lower().translate(translator).strip()

# TF-IDF based summarization
def summarize_with_tfidf(text, query, top_k=3):
    sentences = sent_tokenize(text)
    if len(sentences) == 0:
        return []
    cleaned_sentences = [preprocess(s) for s in sentences]
    cleaned_query = preprocess(query)

    vectorizer = TfidfVectorizer().fit(cleaned_sentences + [cleaned_query])
    sentence_vectors = vectorizer.transform(cleaned_sentences)
    query_vector = vectorizer.transform([cleaned_query])

    similarities = cosine_similarity(query_vector, sentence_vectors).flatten()
    top_indices = similarities.argsort()[-top_k:][::-1]
    return [sentences[i] for i in top_indices]

# Keyword overlap summarization
def summarize_with_keyword_overlap(text, query, top_k=3):
    sentences = sent_tokenize(text)
    if len(sentences) == 0:
        return []
    q_tokens = set(preprocess(query).split())
    scores = []
    for i, s in enumerate(sentences):
        s_tokens = set(preprocess(s).split())
        overlap = len(q_tokens & s_tokens)
        scores.append((overlap, i))
    # sort by overlap desc, then by sentence index to keep original order ties
    scores.sort(key=lambda x: (x[0], -x[1]), reverse=True)
    top = [idx for score, idx in scores[:top_k]]
    return [sentences[i] for i in top]

# Lead-k summarization (first k sentences)
def summarize_with_leadk(text, top_k=3):
    sentences = sent_tokenize(text)
    return sentences[:min(top_k, len(sentences))]

# Widgets UI
text_input = widgets.Textarea(
    value="",
    placeholder="Paste your document here...",
    description="Document:",
    layout=widgets.Layout(width='100%', height='200px')
)

query_input = widgets.Text(
    value="legal",
    placeholder="Enter your query (e.g. legal, privacy, medical)...",
    description="Query:",
    layout=widgets.Layout(width='60%')
)

slider_k = widgets.IntSlider(
    value=3,
    min=1,
    max=10,
    step=1,
    description='# Sentences:'
)

method_dropdown = widgets.Dropdown(
    options=['BM25', 'TF-IDF', 'Lead-k', 'Keyword Overlap'],
    value='BM25',
    description='Method:'
)

run_button = widgets.Button(description="Summarize")
output = widgets.Output()

# Main callback
def on_click(b):
    with output:
        clear_output()
        text = text_input.value.strip()
        query = query_input.value.strip()
        k = slider_k.value
        method = method_dropdown.value

        if not text:
            print("⚠️ Please paste your document.")
            return
        if method != 'Lead-k' and not query:
            print("⚠️ Please enter a query (except for Lead-k which does not require a query).")
            return

        try:
            if method == 'TF-IDF':
                summary = summarize_with_tfidf(text, query, top_k=k)
            elif method == 'BM25':
                # Lazy import for BM25
                from rank_bm25 import BM25Okapi
                sentences = sent_tokenize(text)
                tokenized_sentences = [preprocess(s).split() for s in sentences]
                tokenized_query = preprocess(query).split()
                if len(tokenized_sentences) == 0:
                    summary = []
                else:
                    bm25 = BM25Okapi(tokenized_sentences)
                    scores = bm25.get_scores(tokenized_query)
                    top_indices = np.argsort(scores)[-k:][::-1]
                    summary = [sentences[i] for i in top_indices]
            elif method == 'Lead-k':
                summary = summarize_with_leadk(text, top_k=k)
            elif method == 'Keyword Overlap':
                summary = summarize_with_keyword_overlap(text, query, top_k=k)
            else:
                print("Unknown method selected.")
                return

            print(f"\n🔍 Summary using {method} for query: '{query}' (Top {k} sentences)\n")
            if not summary:
                print("No sentences were returned (document may be empty).")
            else:
                for i, sent in enumerate(summary, 1):
                    print(f"{i}. {sent}")
        except Exception as e:
            print("An error occurred during summarization:", str(e))

run_button.on_click(on_click)

# Display UI
display(text_input, query_input, slider_k, method_dropdown, run_button, output)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Textarea(value='', description='Document:', layout=Layout(height='200px', width='100%'), placeholder='Paste yo…

Text(value='legal', description='Query:', layout=Layout(width='60%'), placeholder='Enter your query (e.g. lega…

IntSlider(value=3, description='# Sentences:', max=10, min=1)

Dropdown(description='Method:', options=('BM25', 'TF-IDF', 'Lead-k', 'Keyword Overlap'), value='BM25')

Button(description='Summarize', style=ButtonStyle())

Output()

In [ ]:
# Colab / Python: End-to-end evaluation with 4 methods on HuggingFace SQuAD (~1000 examples)
# Paste into one Colab cell and run.

# Install dependencies (first cell run)
!pip install --quiet datasets rank_bm25 scikit-learn nltk

# Imports
import nltk
nltk.download('punkt')

from datasets import load_dataset
from nltk.tokenize import sent_tokenize
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np
import string
import csv
from collections import defaultdict
from tqdm import tqdm

# ---------------------------
# Helper utilities
# ---------------------------
translator = str.maketrans("", "", string.punctuation)

def clean_text(t):
    # lowercase and remove punctuation
    return t.lower().translate(translator)

def find_relevant_sentence_indices(context, answer_text):
    """
    Split context into sentences and return indices of sentences that contain the answer_text.
    If no sentence contains the exact answer_text, returns empty list.
    """
    sentences = sent_tokenize(context)
    indices = []
    if not answer_text:
        return sentences, indices
    # Use exact substring match (ground-truth answer is a span from the context)
    for i, s in enumerate(sentences):
        if answer_text in s:
            indices.append(i)
    return sentences, indices

# ---------------------------
# Methods (return indices of top-k sentences)
# ---------------------------

def lead_k_indices(sentences, k):
    return list(range(min(len(sentences), k)))

def keyword_overlap_indices(sentences, query, k):
    q_words = set(clean_text(query).split())
    scores = []
    for i, s in enumerate(sentences):
        s_words = set(clean_text(s).split())
        overlap = len(q_words & s_words)
        scores.append((overlap, i))
    scores.sort(reverse=True, key=lambda x: x[0])
    top = [idx for score, idx in scores[:k]]
    return top

def tfidf_indices(sentences, query, k):
    cleaned_sentences = [clean_text(s) for s in sentences]
    cleaned_query = clean_text(query)
    # If there are no sentences, return empty
    if not cleaned_sentences:
        return []
    # Fit TF-IDF on sentences plus query so query uses same vocab
    vectorizer = TfidfVectorizer().fit(cleaned_sentences + [cleaned_query])
    sentence_vectors = vectorizer.transform(cleaned_sentences)
    query_vector = vectorizer.transform([cleaned_query])
    sims = cosine_similarity(query_vector, sentence_vectors).flatten()
    top = sims.argsort()[-k:][::-1]
    return list(top)

def bm25_indices(sentences, query, k):
    tokenized_sentences = [clean_text(s).split() for s in sentences]
    tokenized_query = clean_text(query).split()
    if not tokenized_sentences:
        return []
    bm25 = BM25Okapi(tokenized_sentences)
    scores = bm25.get_scores(tokenized_query)
    top = np.argsort(scores)[-k:][::-1]
    return list(top)

# ---------------------------
# Evaluation setup
# ---------------------------

methods = {
    "lead_k": lead_k_indices,
    "keyword": keyword_overlap_indices,
    "tfidf": tfidf_indices,
    "bm25": bm25_indices,
}

# Aggregated counters for TP/FP/FN/TN per method
results = {name: {"TP": 0, "FP": 0, "FN": 0, "TN": 0} for name in methods.keys()}

K = 3      # number of sentences to select (change if you want)
N = 1000   # number of examples to evaluate (set to 1000 or less if needed)

print(f"Loading {N} examples from SQuAD validation split...")
dataset = load_dataset("squad", split=f"validation[:{N}]")

skipped = 0
# Optional: write per-example CSV for inspection
csv_rows = []
csv_header = ["id", "method", "selected_indices", "true_indices", "context_snippet"]

for ex_idx, ex in enumerate(tqdm(dataset)):
    context = ex["context"]
    question = ex["question"]
    answers = ex["answers"]["text"]
    answer_text = answers[0] if answers else ""  # use first answer if present

    # split into sentences and find true relevant sentence indices
    sentences, true_indices = find_relevant_sentence_indices(context, answer_text)

    # If we can't locate the answer exactly in any sentence, skip this example
    # (this avoids ambiguous matches and focuses the evaluation on clear cases)
    if len(sentences) == 0 or len(true_indices) == 0:
        skipped += 1
        continue

    # For each method, get predicted indices and update TP/FP/FN/TN
    for method_name, func in methods.items():
        if method_name == "lead_k":
            pred = func(sentences, K)
        else:
            pred = func(sentences, question, K)

        # Make a predicted labels list aligned with sentences
        pred_set = set(pred)
        pred_labels = [1 if i in pred_set else 0 for i in range(len(sentences))]
        true_set = set(true_indices)
        true_labels = [1 if i in true_set else 0 for i in range(len(sentences))]

        # Update counts
        for t, p in zip(true_labels, pred_labels):
            if t == 1 and p == 1:
                results[method_name]["TP"] += 1
            elif t == 0 and p == 1:
                results[method_name]["FP"] += 1
            elif t == 1 and p == 0:
                results[method_name]["FN"] += 1
            else:
                results[method_name]["TN"] += 1

        # For CSV/inspection: record one row per method/example
        csv_rows.append({
            "id": ex_idx,
            "method": method_name,
            "selected_indices": ",".join(map(str, sorted(pred))),
            "true_indices": ",".join(map(str, sorted(true_indices))),
            "context_snippet": context[:200].replace("\n", " ")
        })

print(f"Finished loop. Skipped {skipped} examples where answer-span was not found inside any sentence.")

# ---------------------------
# Compute metrics
# ---------------------------

def compute_metrics_from_counts(counts):
    TP = counts["TP"]
    FP = counts["FP"]
    FN = counts["FN"]
    TN = counts["TN"]
    eps = 1e-9
    precision = TP / (TP + FP + eps)
    recall    = TP / (TP + FN + eps)
    f1        = 2 * precision * recall / (precision + recall + eps)
    accuracy  = (TP + TN) / (TP + FP + FN + TN + eps)
    return precision, recall, f1, accuracy, TP, FP, FN, TN

print("\nMethod     | Precision | Recall  | F1      | Accuracy | TP   FP   FN   TN")
print("-----------+-----------+---------+---------+----------+----------------------")
for name, counts in results.items():
    p, r, f1, acc, TP, FP, FN, TN = compute_metrics_from_counts(counts)
    print(f"{name:<10} | {p:9.3f} | {r:7.3f} | {f1:7.3f} | {acc:8.3f} | {TP:4d} {FP:4d} {FN:4d} {TN:6d}")

# Save CSV for inspection (optional)
csv_file = "eval_predictions.csv"
with open(csv_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=csv_header)
    writer.writeheader()
    for row in csv_rows:
        writer.writerow(row)

print(f"\nSaved predictions to {csv_file}")
print("\nNote: You can change K (top-k sentences) or N (num examples).")
print("Project description file path (uploaded earlier): /mnt/data/Project Description.pdf")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Loading 1000 examples from SQuAD validation split...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

100%|██████████| 1000/1000 [00:03<00:00, 304.61it/s]

Finished loop. Skipped 2 examples where answer-span was not found inside any sentence.

Method     | Precision | Recall  | F1      | Accuracy | TP   FP   FN   TN
-----------+-----------+---------+---------+----------+----------------------
lead_k     |     0.320 |   0.791 |   0.455 |    0.526 |  891 1897  236   1474
keyword    |     0.369 |   0.913 |   0.526 |    0.587 | 1029 1759   98   1612
tfidf      |     0.365 |   0.903 |   0.520 |    0.582 | 1018 1770  109   1601
bm25       |     0.367 |   0.908 |   0.523 |    0.584 | 1023 1765  104   1606

Saved predictions to eval_predictions.csv

Note: You can change K (top-k sentences) or N (num examples).
Project description file path (uploaded earlier): /mnt/data/Project Description.pdf


In [ ]:
# Colab-ready evaluation script
# Paste this into a Colab cell and run.

# Install dependencies
!pip install --quiet datasets rank_bm25 scikit-learn tqdm

# Imports
from datasets import load_dataset
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np
import re
import string
import csv
from tqdm import tqdm
from collections import defaultdict
import sys

# -----------------------------
# Configuration (change if desired)
# -----------------------------
K = 3       # top-k sentences selected per method
N = 1000    # number of SQuAD examples to evaluate (set <= 1000)
DATASET_SPLIT = f"validation[:{N}]"  # SQuAD validation subset

# -----------------------------
# Utility functions
# -----------------------------
translator = str.maketrans("", "", string.punctuation)

def clean_text(t):
    """Lowercase and remove punctuation (simple)."""
    if t is None:
        return ""
    return t.lower().translate(translator)

def split_into_sentences(text):
    """Robust sentence splitter using regex. Avoids NLTK dependency."""
    if text is None:
        return []
    # Normalize whitespace and replace newlines with spaces
    text = text.replace("\n", " ").strip()
    if not text:
        return []
    # Split on sentence-ending punctuation followed by whitespace
    sentences = re.split(r'(?<=[\.\?\!])\s+', text)
    # If no punctuation was found (single long paragraph), split on ';' or long chunks
    if len(sentences) == 1:
        # Break into chunks of ~200-300 characters if very long
        s = sentences[0]
        if len(s) > 400:
            chunks = [s[i:i+300].strip() for i in range(0, len(s), 300)]
            sentences = chunks
    # Final cleanup
    sentences = [s.strip() for s in sentences if s and len(s.strip()) > 0]
    return sentences

def find_relevant_sentence_indices(context, answer_text):
    """
    Return (sentences, indices) where indices are the sentence indices containing the answer_text.
    Uses exact substring match; if not found, returns empty indices.
    """
    sentences = split_into_sentences(context)
    indices = []
    if not answer_text:
        return sentences, indices
    # match exact substring (case-sensitive as in SQuAD) and a fallback to lowercase match
    for i, s in enumerate(sentences):
        if answer_text in s:
            indices.append(i)
    if len(indices) == 0:
        # fallback: try lowercase matching
        a_low = answer_text.lower()
        for i, s in enumerate(sentences):
            if a_low in s.lower():
                indices.append(i)
    return sentences, indices

# -----------------------------
# Methods: return list of indices (selected sentences)
# -----------------------------
def lead_k_indices(sentences, k):
    return list(range(min(len(sentences), k)))

def keyword_overlap_indices(sentences, query, k):
    q_words = set(clean_text(query).split())
    scores = []
    for i, s in enumerate(sentences):
        s_words = set(clean_text(s).split())
        overlap = len(q_words & s_words)
        scores.append((overlap, i))
    scores.sort(reverse=True, key=lambda x: x[0])
    top = [idx for score, idx in scores[:k]]
    return top

def tfidf_indices(sentences, query, k):
    cleaned_sentences = [clean_text(s) for s in sentences]
    cleaned_query = clean_text(query)
    if len(cleaned_sentences) == 0:
        return []
    try:
        vectorizer = TfidfVectorizer().fit(cleaned_sentences + [cleaned_query])
        sentence_vectors = vectorizer.transform(cleaned_sentences)
        query_vector = vectorizer.transform([cleaned_query])
        sims = cosine_similarity(query_vector, sentence_vectors).flatten()
        top = sims.argsort()[-k:][::-1]
        return list(top)
    except Exception as e:
        # fallback: return lead_k
        return lead_k_indices(sentences, k)

def bm25_indices(sentences, query, k):
    tokenized_sentences = [clean_text(s).split() for s in sentences]
    tokenized_query = clean_text(query).split()
    if not tokenized_sentences:
        return []
    bm25 = BM25Okapi(tokenized_sentences)
    scores = bm25.get_scores(tokenized_query)
    top = np.argsort(scores)[-k:][::-1]
    return list(top)

# -----------------------------
# Evaluation loop
# -----------------------------
methods = {
    "lead_k": lead_k_indices,
    "keyword": keyword_overlap_indices,
    "tfidf": tfidf_indices,
    "bm25": bm25_indices,
}

# initialize counters
results = {name: {"TP": 0, "FP": 0, "FN": 0, "TN": 0} for name in methods.keys()}

# Load dataset from Hugging Face
print(f"Loading {N} examples from SQuAD validation split via Hugging Face...")
try:
    dataset = load_dataset("squad", split=DATASET_SPLIT)
except Exception as e:
    print("Failed to load dataset. Error:", e)
    raise

skipped = 0
csv_rows = []
csv_header = ["id", "method", "selected_indices", "true_indices", "context_snippet"]

for ex_idx, ex in enumerate(tqdm(dataset)):
    context = ex.get("context", "")
    question = ex.get("question", "")
    answers = ex.get("answers", {}).get("text", [])
    answer_text = answers[0] if answers else ""
    sentences, true_indices = find_relevant_sentence_indices(context, answer_text)

    # If we can't locate the answer exactly in any sentence, skip (avoid noisy labels)
    if len(sentences) == 0 or len(true_indices) == 0:
        skipped += 1
        continue

    for method_name, func in methods.items():
        if method_name == "lead_k":
            pred = func(sentences, K)
        else:
            pred = func(sentences, question, K)

        pred_set = set(pred)
        pred_labels = [1 if i in pred_set else 0 for i in range(len(sentences))]
        true_set = set(true_indices)
        true_labels = [1 if i in true_set else 0 for i in range(len(sentences))]

        for t, p in zip(true_labels, pred_labels):
            if t == 1 and p == 1:
                results[method_name]["TP"] += 1
            elif t == 0 and p == 1:
                results[method_name]["FP"] += 1
            elif t == 1 and p == 0:
                results[method_name]["FN"] += 1
            else:
                results[method_name]["TN"] += 1

        csv_rows.append({
            "id": ex_idx,
            "method": method_name,
            "selected_indices": ",".join(map(str, sorted(pred))),
            "true_indices": ",".join(map(str, sorted(true_indices))),
            "context_snippet": context[:200].replace("\n", " ")
        })

print(f"Done. Skipped {skipped} examples where answer span not found in any sentence.")
# -----------------------------
# Compute metrics and print table
# -----------------------------
def compute_metrics(counts):
    TP = counts["TP"]; FP = counts["FP"]; FN = counts["FN"]; TN = counts["TN"]
    eps = 1e-9
    precision = TP / (TP + FP + eps)
    recall = TP / (TP + FN + eps)
    f1 = 2 * precision * recall / (precision + recall + eps)
    accuracy = (TP + TN) / (TP + FP + FN + TN + eps)
    return precision, recall, f1, accuracy, TP, FP, FN, TN

print("\nMethod     | Precision | Recall  | F1      | Accuracy | TP   FP   FN   TN")
print("-----------+-----------+---------+---------+----------+----------------------")
for name, counts in results.items():
    p, r, f1, acc, TP, FP, FN, TN = compute_metrics(counts)
    print(f"{name:<10} | {p:9.3f} | {r:7.3f} | {f1:7.3f} | {acc:8.3f} | {TP:4d} {FP:4d} {FN:4d} {TN:6d}")

# Save CSV
csv_file = "eval_predictions.csv"
with open(csv_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=csv_header)
    writer.writeheader()
    for row in csv_rows:
        writer.writerow(row)

print(f"\nSaved per-example predictions to '{csv_file}' in the current working directory.")
print("You can change K (top-k) or N (num examples) at the top of the script and re-run.")


Loading 1000 examples from SQuAD validation split via Hugging Face...


100%|██████████| 1000/1000 [00:03<00:00, 303.62it/s]

Done. Skipped 6 examples where answer span not found in any sentence.

Method     | Precision | Recall  | F1      | Accuracy | TP   FP   FN   TN
-----------+-----------+---------+---------+----------+----------------------
lead_k     |     0.318 |   0.782 |   0.452 |    0.538 |  879 1886  245   1599
keyword    |     0.370 |   0.910 |   0.526 |    0.600 | 1023 1742  101   1743
tfidf      |     0.366 |   0.901 |   0.521 |    0.596 | 1013 1752  111   1733
bm25       |     0.368 |   0.906 |   0.524 |    0.598 | 1018 1747  106   1738

Saved per-example predictions to 'eval_predictions.csv' in the current working directory.
You can change K (top-k) or N (num examples) at the top of the script and re-run.
